In [60]:

from nsepy import get_history
from datetime import datetime,timedelta,date
import pandas as pd
import numpy as np
import datetime
import matplotlib
import operator
import itertools
from nsepy.symbols import get_symbol_list, get_index_constituents_list
import matplotlib.pyplot as plt
%matplotlib inline


def download_data(token_name, from_date, to_date,interval,indexbool,l):
    i=0
    while i < len(token_name):
        records = get_history(symbol=token_name[i],start=from_date,end=to_date,index=indexbool)
        df = pd.DataFrame(records)
        if i ==0:
            df_final = df
        else:
            df_final = pd.concat([df_final, df],axis = 0) 
        i = i+1
    df_final['Date']=df_final.index
    df_final.to_excel(str(l)+'_rawlong.xlsx')

def read_data(filename,token_name,from_date, to_date,interval,indexbool,lookback,holdingdays):
    df = pd.read_excel(filename,index_col=0)
    i=0
    while i < len(token_name):
        df1=df[df['Symbol']==token_name[i]]
        df1['Change'] = (df1.Close - df1.Close.shift(lookback))*100/df1.Close.shift(lookback)
        df1['afterholding']=df1['Close'].pct_change(holdingdays)
        if i ==0:
            df_final = df1
        else:
            df_final = pd.concat([df_final, df1],axis = 0) 
        i = i+1
    df_final['Date']=df_final.index
    return df_final
    
    
def getdata(token_name, from_date, to_date,interval,indexbool,lookback):
    i=0
    while i < len(token_name):
        records = get_history(symbol=token_name[i],start=from_date,end=to_date,index=indexbool)
        df = pd.DataFrame(records)
        df['Change'] = (df.Close - df.Close.shift(lookback))*100/df.Close.shift(lookback)
        #df['Mean'] = df.Close.shift(1).rolling(50).mean()
        #df['U_BB'] = df.Close.shift(1).rolling(50).mean()+ 2*df.Close.shift(1).rolling(50).std()
        #df['L_BB'] = df.Close.shift(1).rolling(50).mean()- 2*df.Close.shift(1).rolling(50).std()
        if i ==0:
            df_final = df
        else:
            df_final = pd.concat([df_final, df],axis = 0) 
        i = i+1
    df_final['Date']=df_final.index
    return df_final    


def createbasket(df_final,basket_elements,indexbool,lookback,holdingdays):
    output=pd.pivot_table(df_final,index=df_final.index,columns='Symbol',values=["Change"])
    output=output.fillna(0)
    pivotdicts=output.to_dict(orient="records")
    wt = 1.0/basket_elements
    basket_df=pd.DataFrame()
    #datelist=[]
    basketlist=[]
    for i in range(lookback,len(output),holdingdays):
        sorted_pivotdicts=dict(sorted(pivotdicts[i].items(), key=operator.itemgetter(1),reverse=True))
        temp_dict={'basket':dict(itertools.islice(sorted_pivotdicts.items(), basket_elements))}
        temp_list=[]
        #datelist.append()
        basket_dict={}
        for key in temp_dict['basket'].keys():
            temp_list.append(key[1])
            temp_tup=('Change',key[1])
            basket_dict[key[1]]=temp_dict['basket'][temp_tup]
            del temp_tup
    
        current_date=output.index[i]
        prev_date=output.index[i-lookback]
        for token in temp_list:
            #print(token)
            temp_df=df_final[df_final['Symbol']==token]
            temp_df_curr=temp_df[temp_df['Date']==current_date]
            temp_df_prev=temp_df[temp_df['Date']==prev_date]
            current_close=temp_df_curr['Close']
            prev_close=temp_df_prev['Close']
            #print(current_close,prev_close)
            if not indexbool:
                try:
                    ret = (current_close.iloc[0]-prev_close.iloc[0])/100.0
                    if (ret<=0.0):
                        del basket_dict[token]
                except:
                    pass

        basketlist.append({'basket':basket_dict})
    basket_df=pd.DataFrame(basketlist,index=output.index.tolist()[lookback::holdingdays])
    basket_df['Date']=basket_df.index
    return basket_df


def getbasketreturn(df_final,basket_df,holdingdays,basket_elements):
    return_df=pd.DataFrame()
    wt = 1.0/basket_elements
#     test=df_final['Close']
#     df_final['afterholding']=test.pct_change(holdingdays)
    print(df_final.head(20))
    basket_df['afterholding']=''
    for i in range(0,len(basket_df)):
        tokennames=[]
        change=pd.DataFrame()
        for key in basket_df.iloc[i,0].keys():
            tokennames.append(key)
        dt=basket_df.index.to_list()[i]
        dt=pd.to_datetime(dt, format='%Y-%m-%d', errors='ignore')
        #print(dt)
        portfolio=getweights(tokennames,df_final,holdingdays,dt)
        print(portfolio)
        cond1 = df_final['Symbol'].isin(tokennames)
        cond2 = df_final['Date']==basket_df.iloc[i,1]
        print(basket_df.iloc[i,1])
        
        change=df_final[cond1 & cond2][['Symbol','afterholding']]
        #change=change.fillna(0)
        print(change)
        returnn=0
        for idx,row in change.iterrows():
            returnn=returnn+portfolio[row['Symbol']]*row['afterholding']
        #for ret in change.tolist():
        #    returnn=returnn+wt*ret
        print(returnn)
        basket_df['afterholding'][i]=returnn*100
    #print(basket_df)
    return_df=basket_df[basket_df['afterholding']!= ""]
    return return_df

def getweights(tokennames,df_final,holdingdays,dt):
    #df = df_final[df_final['Symbol'].isin(tokennames)]
    df=pd.DataFrame()
    ann = 252/holdingdays
    for token in tokennames:
        l = df_final[df_final['Symbol']==token].Close
        df[token]=l
    df.index = pd.to_datetime(df.index)
    df=df.loc[:dt]
    #print(df)
    p_ret = [] # Define an empty array for portfolio returns
    p_vol = [] # Define an empty array for portfolio volatility
    p_weights = [] # Define an empty array for asset weights
    num_assets = len(df.columns)
    #print(num_assets)
    num_portfolios = 10
    for portfolio in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights = weights/np.sum(weights)
        p_weights.append(weights)
        ind_er = df.resample(str(holdingdays)+'D').last().pct_change().mean()
        ind_er=ind_er.fillna(0)
        
        returns = np.dot(weights, ind_er)
        p_ret.append(returns)
        cov_matrix = df.pct_change().apply(lambda x: np.log(1+x)).cov()
        var = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()# Portfolio Variance
        sd = np.sqrt(var) # Daily standard deviation
        #change. here
        ann_sd = sd*np.sqrt(ann) # Annual standard deviation = volatility
        p_vol.append(ann_sd)
    data = {'Returns':p_ret, 'Volatility':p_vol}

    for counter, symbol in enumerate(df.columns.tolist()):
        #print(counter, symbol)
        data[symbol] = [w[counter] for w in p_weights]
    portfolios  = pd.DataFrame(data)
    #print(portfolios)
    #portfolios.plot.scatter(x='Volatility', y='Returns', marker='o', s=10, alpha=0.3, grid=True, figsize=[10,10])
    min_vol_port = portfolios.iloc[portfolios['Volatility'].idxmin()]
    #print(min_vol_port)
    # Finding the optimal portfolio
    rf = 0.01 # risk factor
    optimal_risky_port = portfolios.iloc[((portfolios['Returns']-rf)/portfolios['Volatility']).idxmax()]
    # Plotting optimal portfolio
#     plt.subplots(figsize=(10, 10))
#     plt.scatter(portfolios['Volatility'], portfolios['Returns'],marker='o', s=10, alpha=0.3)
#     plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=50)
#     plt.scatter(optimal_risky_port[1], optimal_risky_port[0], color='g', marker='*', s=50)
    return optimal_risky_port
   


In [ ]:
download_data(indexwise_dict['nifty50'], from_date, to_date,interval,False,'nifty50')

In [ ]:
read_data('nifty50_raw.xlsx',from_date, to_date,interval,False,2)

In [2]:
#refer for keyword:https://www1.nseindia.com/products/content/equities/indices/broad_indices.htm
nifty50=get_index_constituents_list("nifty50").Symbol.tolist()
niftymidcap50=get_index_constituents_list("niftymidcap50").Symbol.tolist()
niftymidcap100=get_index_constituents_list("niftymidcap100").Symbol.tolist()
niftymidcap150=get_index_constituents_list("niftymidcap150").Symbol.tolist()
niftysmallcap100=get_index_constituents_list("niftysmallcap100").Symbol.tolist()
token_name = ['ADANIPOWER','AMARAJABAT','APOLLOHOSP','APOLLOTYRE','ASHOKLEY','BALKRISIND','BANKINDIA','BATAINDIA','BEL','BHARATFORG','BHEL','CESC','CANBK','CASTROLIND','CHOLAFIN','CUMMINSIND','ESCORTS','EXIDEIND','FEDERALBNK','GMRINFRA','GLENMARK','GODREJPROP','IDFCFIRSTB','IBULHSGFIN','JINDALSTEL','JUBLFOOD','L&TFH','LICHSGFIN','MRF','MGL','MANAPPURAM','MFSL','MINDTREE','NATIONALUM','OIL','RBLBANK','RECLTD','SRF','SAIL','SUNTV','TVSMOTOR','TATACONSUM','TATAPOWER','RAMCOCEM','TORNTPOWER','UNIONBANK','IDEA','VOLTAS','NETFMID150']
index_name=['NIFTYMID50','NIFTY SMALLCAP 50','NIFTY AUTO','NIFTY BANK','NIFTY IT','NIFTY METAL','NIFTY REALTY','NIFTY PHARMA','NIFTY FMCG','NIFTY GROWSECT 15','NIFTY50 VALUE 20','NIFTY GS 10YR']
etf_name=['GOLDBEES','LIQUIDBEES','LICNETFGSC','SETF10GILT','NETFLTGILT']


indexwise_dict={}
indexwise_dict['nifty50']=nifty50
#indexwise_dict['niftymidcap50']=niftymidcap50
#indexwise_dict['niftymidcap100']=niftymidcap100
#indexwise_dict['niftymidcap150']=niftymidcap150
#indexwise_dict['niftysmallcap100']=niftysmallcap100
#indexwise_dict['etfs']=etf_name




In [ ]:
to_date = date.today()- timedelta(days=0)
from_date = to_date - timedelta(days=100)
#from_date = date(2015,1,1)
interval = "day"#"5minute"

# lookback=5
# holdingdays=5
# basket_elements=25

#from 2015
weekly={'lookback':[10,20,40,80,100,200],'holdingdays':5,'basket_elements':25,'from_date':date(2015,1,1)}
biweekly={'lookback':[20,40,80,160,320,640],'holdingdays':10,'basket_elements':25,'from':2*100}
monthly={'lookback':[40,80,160,320,640,1280],'holdingdays':20,'basket_elements':25,'from'=4*100}

quarterly={'lookback':5,'holdingdays':60,'basket_elements':25,'from':3*4*100}
semianually={'lookback':5,'holdingdays':120,'basket_elements':25,'from':6*4*100}
annually={'lookback':5,'holdingdays':240,'basket_elements':25,'from':12*4*100}

output_df=pd.DataFrame()

In [ ]:
for l in indexwise_dict.keys():
    print(l)
    df_final = getdata(indexwise_dict[l], from_date, to_date,interval,False,lookback)
    basket_df=createbasket(df_final,basket_elements,False)
    return_df=getbasketreturn(df_final,basket_df,holdingdays,basket_elements)
    output_df[l]=return_df['afterholding'].apply(lambda x: (x)/100 +1 )
print(output_df)

In [ ]:
#--------------Comparing portfolios with different lookback periods for weekly rebalancing

to_date = date.today()- timedelta(days=0)
interval = "day"
from_date=date(2000,1,1)

for l in indexwise_dict.keys():
    download_data(indexwise_dict[l], from_date, to_date,interval,False,l)
    




In [ ]:
years=(to_date-date(2015,1,1))
years.days/365

In [ ]:
records = get_history(symbol='IOC',start=date(2000,1,1),end=date(2010,1,1),index=False)

In [ ]:
records

In [7]:
#20,40,80,100,200
#weekly={'lookback':[10,20,40,80,160],'holdingdays':5,'basket_elements':25,'from_date':date(2015,1,1)}
#biweekly={'lookback':[20,40,80,160,320],'holdingdays':10,'basket_elements':25,'from':date(2015,1,1)}
monthly={'lookback':[40,80,160,320,640],'holdingdays':20,'basket_elements':25,'from':date(2015,1,1)}
#-----------------
#medium term

#weekly: 5,10,20,60,120
#biweekly:5,10,20,60,120
#monthly:10,20,60,120,240

#long term
#from:2009
# quarterly:20,60,120,240
# halfyearly:60,120,240
# yearly:120,240

#-----------------
from_date=date(2015,1,1)
to_date = date.today()- timedelta(days=0)
interval = "day"
holdingdays=10
basket_elements=10
output_df=pd.DataFrame()
returns={}
sharpe_ratios={}
max_drawdown={}
std_devs={}

for l in indexwise_dict.keys():
    for lookback in [5]:
        df_final = read_data(str(l)+'_raw.xlsx', from_date, to_date,interval,False,lookback)
        print(l, lookback)
        basket_df=createbasket(df_final,basket_elements,False,lookback)
        print('baskets generated')
        return_df=getbasketreturn(df_final,basket_df,holdingdays,basket_elements)
        print('got the returns')
        output_df[l+','+str(lookback)+' days']=return_df['afterholding'].apply(lambda x: (x)/100 +1 )
print(output_df)

nifty50 5
baskets generated


/Users/akshaykulkarni/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/akshaykulkarni/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Users/akshaykulkarni/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/Users/akshaykulkarni/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7605: RuntimeWarning: Degrees of freedom <= 0 for slice
  baseCov = np.cov(mat.T)
/Users/akshaykulkarni/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divid

got the returns
            nifty50,5 days
2015-01-08        1.009741
2015-01-22        1.002037
2015-02-06        1.006261
2015-02-23        0.987089
2015-03-09        0.962842
...                    ...
2020-11-03        1.132771
2020-11-17        1.075495
2020-12-02        1.064402
2020-12-16        1.000161
2020-12-31        1.037844

[149 rows x 1 columns]


In [61]:
from_date=date(2015,1,1)
to_date = date.today()- timedelta(days=0)
interval = "day"
holdingdays=10
basket_elements=10
output_df=pd.DataFrame()
returns={}
sharpe_ratios={}
max_drawdown={}
std_devs={}

for l in indexwise_dict.keys():
    for lookback in [5]:
        df_final = read_data(str(l)+'_raw.xlsx',indexwise_dict[l], from_date, to_date,interval,False,lookback,holdingdays)
        print(l, lookback)
#         df_final=df_final.replace(np.nan,0)
        #print(df_final.head())
        basket_df=createbasket(df_final,basket_elements,False,lookback,holdingdays)
        print(basket_df)
        print('baskets generated')
        return_df=getbasketreturn(df_final,basket_df,holdingdays,basket_elements)
        print('got the returns')
        output_df[l+','+str(lookback)+' days']=return_df['afterholding'].apply(lambda x: (x)/100 +1 )
print(output_df)

/Users/akshaykulkarni/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/akshaykulkarni/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


nifty50 5
                                                       basket       Date
2015-01-15  {'ULTRACEMCO': 13.037819208418133, 'HINDUNILVR... 2015-01-15
2015-01-30  {'BPCL': 13.971539456662367, 'BAJAJFINSV': 11.... 2015-01-30
2015-02-13  {'ADANIPORTS': 11.708860759493684, 'DRREDDY': ... 2015-02-13
2015-02-28  {'AXISBANK': 11.574537838083977, 'NTPC': 6.744... 2015-02-28
2015-03-16  {'BHARTIARTL': 13.087785703887027, 'INFY': 3.4... 2015-03-16
...                                                       ...        ...
2020-11-10  {'INDUSINDBK': 29.167955459325707, 'BAJFINANCE... 2020-11-10
2020-11-24  {'BAJAJFINSV': 20.28892041790275, 'M&M': 14.45... 2020-11-24
2020-12-09  {'UPL': 11.460572462948306, 'SBIN': 9.47560234... 2020-12-09
2020-12-23  {'WIPRO': 7.575334821428582, 'INFY': 7.4933516... 2020-12-23
2021-01-07  {'HINDALCO': 13.4483475368946, 'TATASTEEL': 12... 2021-01-07

[149 rows x 2 columns]
baskets generated
                Symbol Series  Prev Close    Open    High     Low    Las

/Users/akshaykulkarni/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Returns       0.059623
Volatility    0.056954
BPCL          0.003879
BAJAJFINSV    0.002238
TITAN         0.012745
HCLTECH       0.252636
EICHERMOT     0.089053
CIPLA         0.113731
HDFCBANK      0.161509
BAJFINANCE    0.220129
DIVISLAB      0.077998
ITC           0.066083
Name: 4, dtype: float64
2015-01-30 00:00:00
                Symbol  afterholding
Date                                
2015-01-30  BAJFINANCE      0.049311
2015-01-30  BAJAJFINSV      0.122959
2015-01-30        BPCL      0.129572
2015-01-30       CIPLA      0.090432
2015-01-30    DIVISLAB      0.027571
2015-01-30   EICHERMOT      0.075933
2015-01-30     HCLTECH      0.124588
2015-01-30    HDFCBANK      0.084781
2015-01-30         ITC      0.029757
2015-01-30       TITAN      0.145517
0.07981923047655096


/Users/akshaykulkarni/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

/Users/akshaykulkarni/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Returns       0.002625
Volatility    0.072332
WIPRO         0.053203
INFY          0.047511
CIPLA         0.038651
HCLTECH       0.185597
DRREDDY       0.133269
TCS           0.090137
TECHM         0.154259
ASIANPAINT    0.168745
NESTLEIND     0.086545
HINDUNILVR    0.042082
Name: 8, dtype: float64
2020-12-23 00:00:00
                Symbol  afterholding
Date                                
2020-12-23  ASIANPAINT      0.047675
2020-12-23       CIPLA      0.081650
2020-12-23     DRREDDY      0.033064
2020-12-23     HCLTECH      0.059790
2020-12-23  HINDUNILVR      0.034094
2020-12-23        INFY      0.066244
2020-12-23   NESTLEIND      0.066227
2020-12-23         TCS      0.034848
2020-12-23       TECHM      0.020065
2020-12-23       WIPRO      0.072462
0.04710901203029263


/Users/akshaykulkarni/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Returns       0.003323
Volatility    0.080873
HINDALCO      0.159476
TATASTEEL     0.208715
GAIL          0.013659
AXISBANK      0.109376
GRASIM        0.104452
TATAMOTORS    0.025436
BHARTIARTL    0.191534
INDUSINDBK    0.032326
ADANIPORTS    0.028714
TCS           0.126311
Name: 5, dtype: float64
2021-01-07 00:00:00
                Symbol  afterholding
Date                                
2021-01-07  ADANIPORTS      0.091556
2021-01-07    AXISBANK      0.132754
2021-01-07  BHARTIARTL      0.075020
2021-01-07        GAIL      0.128541
2021-01-07      GRASIM      0.101829
2021-01-07    HINDALCO      0.155377
2021-01-07  INDUSINDBK      0.113053
2021-01-07         TCS      0.042647
2021-01-07  TATAMOTORS      0.162825
2021-01-07   TATASTEEL      0.163180
0.11592992803866137
got the returns
            nifty50,5 days
2015-01-15        1.106585
2015-01-30        1.079819
2015-02-13        1.010260
2015-02-28        1.051716
2015-03-16        0.994806
...                    ...
2020-11-10 

/Users/akshaykulkarni/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [50]:
output_df.to_excel('monthly_lookback11.xlsx')

In [ ]:
output_df1=output_df.dropna()
for column in output_df1.columns.to_list():
    sharpe_ratios[column]=(output_df1[column]-1).mean()/(output_df1[column]-1).std()
    max_drawdown[column]=(output_df1[column]-1).min()
    returns[column]=output_df1[column].prod()
    std_devs[column]=(output_df1[column]-1).std()




In [ ]:
output_df.describe()


In [ ]:
returns['metric']='returns'
sharpe_ratios['metric']='sharpe_ratios'
max_drawdown['metric']='max_drawdown'
std_devs['metric']='std_devs'
comparision_df=pd.DataFrame()
comparision_df=comparision_df.append(returns,ignore_index=True)
comparision_df=comparision_df.append(sharpe_ratios,ignore_index=True)
comparision_df=comparision_df.append(max_drawdown,ignore_index=True)
comparision_df=comparision_df.append(std_devs,ignore_index=True)
comparision_df.index=comparision_df['metric']
comparision_df.T

In [ ]:
df=output_df
#df=df[df.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:

output2=df
returns={}
sharpe_ratios={}
max_drawdown={}
std_devs={}
returns['metric']='returns'
sharpe_ratios['metric']='sharpe_ratios'
max_drawdown['metric']='max_drawdown'
std_devs['metric']='std_devs'
for column in output2.columns.to_list():
    sharpe_ratios[column]=(output2[column]-1).mean()/(output2[column]-1).std()
    max_drawdown[column]=(output2[column]-1).min()
    returns[column]=output2[column].prod()
    std_devs[column]=(output2[column]-1).std()

comparision_df=pd.DataFrame()
comparision_df=comparision_df.append(returns,ignore_index=True)
comparision_df=comparision_df.append(sharpe_ratios,ignore_index=True)
comparision_df=comparision_df.append(max_drawdown,ignore_index=True)
comparision_df=comparision_df.append(std_devs,ignore_index=True)
comparision_df.index=comparision_df['metric']
comparision_df.T


In [ ]:
returns

In [ ]:
#-------------comparing portfolios with different varying basket_elements
weekly={'lookback':160,'holdingdays':5,'basket_elements':25,'from_date':date(2015,1,1)}
#biweekly={'lookback':[20,40,80,160,320],'holdingdays':10,'basket_elements':25,'from':date(2015,1,1)}
#monthly={'lookback':[40,80,160,320,640],'holdingdays':20,'basket_elements':25,'from':date(2015,1,1)}
holdingdays=weekly['holdingdays']
lookback=weekly['lookback']
to_date = date.today()- timedelta(days=0)
interval = "day"
from_date=date(2015,1,1)
#basket_elements=monthly['basket_elements']
output_df=pd.DataFrame()
basket_element_list=[5,10,25]

for l in indexwise_dict.keys():
    df_final = read_data(str(l)+'_raw.xlsx', from_date, to_date,interval,False,lookback)
    for basket_elements in basket_element_list:
        print(l,basket_elements)
        basket_df=createbasket(df_final,basket_elements,False)
        print('baskets generated')
        return_df=getbasketreturn(df_final,basket_df,holdingdays,basket_elements)
        print('returns generated')
        output_df[l +"_"+ str(basket_elements)]=return_df['afterholding'].apply(lambda x: (x)/100 +1 )


In [ ]:
output_df['M100']=o1
new_df=output_df.cumprod()
new_df.plot()


In [ ]:
new_df